In [ ]:
import re
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold, ParameterGrid, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from scipy.stats import randint, uniform

!pip install imblearn
from imblearn.over_sampling import SMOTE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
class DataPreparation:
    """
    Classe responsável pela preparação e análise exploratória dos dados do Titanic.
    """

    def __init__(self, url):
        """
        Inicializa a classe carregando os dados do CSV a partir da URL fornecida.

        :param url: URL do arquivo CSV contendo os dados do Titanic.
        """
        self.df = pd.read_csv(url)

    def descriptive_analysis(self):
        """
        Realiza uma análise descritiva dos dados, incluindo estatísticas resumidas e visualizações.
        """
        # Exibe estatísticas descritivas do DataFrame
        print(self.df.describe())

        # Visualiza a distribuição de idades
        sns.histplot(self.df['Age'].dropna(), kde=True)
        plt.title('Distribuição de Idades')
        plt.show()

        # Cria um boxplot de idade por classe
        sns.boxplot(x='Pclass', y='Age', data=self.df)
        plt.title('Boxplot de Idade por Classe')
        plt.show()

        # Gera uma matriz de correlação para variáveis numéricas
        numeric_columns = self.df.select_dtypes(include=[np.number])
        sns.heatmap(numeric_columns.corr(), annot=True, cmap="coolwarm")
        plt.title('Matriz de Correlação')
        plt.show()

        # Chama métodos adicionais para análises específicas
        self.plot_survival_by_gender()
        self.plot_survival_by_class()
        self.plot_age_distribution_by_survival()

    def plot_survival_by_gender(self):
        """
        Plota um gráfico de contagem mostrando a sobrevivência por gênero.
        """
        sns.countplot(x='Survived', hue='Sex_male', data=self.df)
        plt.title('Sobrevivência por Gênero')
        plt.xlabel('Sobreviveu')
        plt.ylabel('Contagem')
        plt.legend(title='Gênero', labels=['Feminino', 'Masculino'])
        plt.show()

    def plot_survival_by_class(self):
        """
        Plota um gráfico de contagem mostrando a sobrevivência por classe de passageiro.
        """
        sns.countplot(x='Survived', hue='Pclass', data=self.df)
        plt.title('Sobrevivência por Classe')
        plt.xlabel('Sobreviveu')
        plt.ylabel('Contagem')
        plt.legend(title='Classe', loc='upper right')
        plt.show()

    def plot_age_distribution_by_survival(self):
        """
        Plota um boxplot mostrando a distribuição de idades por status de sobrevivência.
        """
        sns.boxplot(x='Survived', y='Age', data=self.df)
        plt.title('Distribuição de Idades por Status de Sobrevivência')
        plt.xlabel('Sobreviveu')
        plt.ylabel('Idade')
        plt.show()

    def preprocess_data(self):
        """
        Realiza o pré-processamento dos dados, incluindo engenharia de features,
        tratamento de valores ausentes, codificação de variáveis categóricas e
        balanceamento de classes.

        :return: X_train, X_test, y_train, y_test - Conjuntos de dados de treino e teste.
        """
        # Engenharia de features
        self.df['FamilySize'] = self.df['SibSp'] + self.df['Parch'] + 1
        self.df['AgeGroup'] = pd.cut(self.df['Age'], bins=[0, 12, 18, 30, 50, 80], labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior'])
        self.df['Fare'] = np.log1p(self.df['Fare'])
        self.df['Title'] = self.df['Name'].apply(self.get_title)
        self.df['FamilyType'] = self.df['FamilySize'].apply(self.categorize_family)
        self.df['AgeFare'] = self.df['Age'] * self.df['Fare']
        self.df['DeckCategory'] = self.df['Cabin'].apply(self.get_deck_category)
        self.df['IsAlone'] = (self.df['FamilySize'] == 1).astype(int)

        # Tratamento de valores ausentes
        numeric_columns = self.df.select_dtypes(include=[np.number]).columns
        categorical_columns = self.df.select_dtypes(include=['object', 'category']).columns

        # Imputação para colunas numéricas
        numeric_imputer = SimpleImputer(strategy='median')
        self.df[numeric_columns] = numeric_imputer.fit_transform(self.df[numeric_columns])

        # Imputação para colunas categóricas
        categorical_imputer = SimpleImputer(strategy='most_frequent')
        self.df[categorical_columns] = categorical_imputer.fit_transform(self.df[categorical_columns])

        # Remove colunas irrelevantes
        self.df.drop(columns=['Cabin', 'Name', 'Ticket'], inplace=True)

        # Codificação one-hot para variáveis categóricas
        categorical_columns = ['Sex', 'Embarked', 'AgeGroup', 'Title', 'FamilyType', 'DeckCategory']
        self.df = pd.get_dummies(self.df, columns=categorical_columns, drop_first=True)

        # Plotagem das novas features após a codificação
        # self.plot_new_features()

        # Separa features e target
        X = self.df.drop(columns=['Survived'])
        y = self.df['Survived']

        # Escalonamento das features numéricas
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        scaler = StandardScaler()
        X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

        # Balanceamento de classes usando SMOTE
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)

        # Divisão em conjuntos de treino e teste
        return train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    def get_title(self, name):
        """
        Extrai o título do nome do passageiro.

        :param name: Nome completo do passageiro.
        :return: Título extraído ou 'Unknown' se não encontrado.
        """
        title_search = re.search(' ([A-Za-z]+)\.', name)
        return title_search.group(1) if title_search else 'Unknown'

    def categorize_family(self, family_size):
        """
        Categoriza o tamanho da família.

        :param family_size: Tamanho da família.
        :return: Categoria do tamanho da família.
        """
        if family_size == 1:
            return 'Alone'
        elif family_size <= 3:
            return 'Small'
        elif family_size <= 6:
            return 'Medium'
        else:
            return 'Large'

    def get_deck_category(self, cabin):
        """
        Obtém a categoria do convés a partir da cabine.

        :param cabin: Número da cabine.
        :return: Categoria do convés ou 'Unknown' se não disponível.
        """
        return 'Unknown' if pd.isna(cabin) else cabin[0]

    def plot_new_features(self):
        # Title
        plt.figure(figsize=(12, 6))
        sns.countplot(x='Title', hue='Survived', data=self.df)
        plt.title('Sobrevivência por Título')
        plt.xticks(rotation=45)
        plt.show()

        # FamilyType
        plt.figure(figsize=(10, 6))
        sns.countplot(x='FamilyType', hue='Survived', data=self.df)
        plt.title('Sobrevivência por Tipo de Família')
        plt.show()

        # AgeFare
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x='Age', y='Fare', hue='Survived', data=self.df)
        plt.title('Relação entre Idade, Tarifa e Sobrevivência')
        plt.show()

        # DeckCategory
        plt.figure(figsize=(10, 6))
        sns.countplot(x='DeckCategory', hue='Survived', data=self.df)
        plt.title('Sobrevivência por Categoria de Convés')
        plt.show()

        # IsAlone
        plt.figure(figsize=(8, 6))
        sns.countplot(x='IsAlone', hue='Survived', data=self.df)
        plt.title('Sobrevivência de Passageiros Sozinhos vs. Acompanhados')
        plt.show()


class PassengerSearch:
    """
    Classe para realizar buscas de passageiros no dataset do Titanic.
    """

    def __init__(self, df):
        """
        Inicializa a classe com o DataFrame dos passageiros.

        :param df: DataFrame contendo os dados dos passageiros.
        """
        self.df = df

    def search_by_id(self, passenger_id):
        """
        Busca um passageiro pelo ID.

        :param passenger_id: ID do passageiro.
        :return: DataFrame com os dados do passageiro encontrado.
        """
        return self.df[self.df['PassengerId'] == passenger_id]

    def search_by_survival(self, survived):
        """
        Busca passageiros pelo status de sobrevivência.

        :param survived: Status de sobrevivência (0 ou 1).
        :return: DataFrame com os passageiros que correspondem ao status.
        """
        return self.df[self.df['Survived'] == survived]

    def search_by_class(self, pclass):
        """
        Busca passageiros pela classe.

        :param pclass: Classe do passageiro (1, 2 ou 3).
        :return: DataFrame com os passageiros da classe especificada.
        """
        return self.df[self.df['Pclass'] == pclass]

def passenger_search_menu(df):
    """
    Função que implementa o menu de busca de passageiros.

    :param df: DataFrame contendo os dados dos passageiros.
    """
    search = PassengerSearch(df)

    while True:
        print("\nMenu de Busca de Passageiros:")
        print("1. Buscar por ID do Passageiro")
        print("2. Buscar por Status de Sobrevivência")
        print("3. Buscar por Classe do Passageiro")
        print("4. Voltar ao Menu Principal")

        choice = input("Escolha uma opção: ")

        if choice == '1':
            passenger_id = int(input("Digite o ID do Passageiro: "))
            print(search.search_by_id(passenger_id))
        elif choice == '2':
            survived = int(input("Digite 1 para sobreviventes ou 0 para não sobreviventes: "))
            print(search.search_by_survival(survived))
        elif choice == '3':
            pclass = int(input("Digite a Classe do Passageiro (1, 2 ou 3): "))
            print(search.search_by_class(pclass))
        elif choice == '4':
            print("Voltando ao Menu Principal...")
            break
        else:
            print("Opção inválida! Por favor, escolha novamente.")

class ModelTraining:
    """
    Classe responsável pelo treinamento de modelos de machine learning.
    """

    def __init__(self, X_train, y_train, k_folds=10):
        """
        Inicializa a classe com os dados de treino e o número de folds para validação cruzada.

        :param X_train: Features de treino.
        :param y_train: Target de treino.
        :param k_folds: Número de folds para validação cruzada.
        """
        self.X_train = X_train
        self.y_train = y_train
        self.k_folds = k_folds

    def decision_tree(self):
        """
        Treina um modelo de Árvore de Decisão e realiza validação cruzada.

        :return: Modelo de Árvore de Decisão treinado.
        """
        dt = DecisionTreeClassifier(random_state=42)
        scores = cross_val_score(dt, self.X_train, self.y_train, cv=self.k_folds, scoring='accuracy')
        print(f"Decision Tree CV Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
        dt.fit(self.X_train, self.y_train)
        return dt

    def random_forest(self):
        """
        Treina um modelo de Random Forest e realiza validação cruzada.

        :return: Modelo de Random Forest treinado.
        """
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        scores = cross_val_score(rf, self.X_train, self.y_train, cv=self.k_folds, scoring='accuracy')
        print(f"Random Forest CV Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
        rf.fit(self.X_train, self.y_train)
        return rf

class HyperparameterTuning:
    """
    Classe para realizar o ajuste de hiperparâmetros dos modelos.
    """

    def __init__(self, model, param_distributions, X_train, y_train, k_folds=5, n_iter=20):
        """
        Inicializa a classe com o modelo, distribuições de parâmetros e dados de treino.

        :param model: Modelo a ser otimizado.
        :param param_distributions: Distribuições de parâmetros para a busca aleatória.
        :param X_train: Features de treino.
        :param y_train: Target de treino.
        :param k_folds: Número de folds para validação cruzada.
        :param n_iter: Número de iterações para a busca aleatória.
        """
        self.model = model
        self.param_distributions = param_distributions
        self.X_train = X_train
        self.y_train = y_train
        self.k_folds = k_folds
        self.n_iter = n_iter

    def random_search(self):
        """
        Realiza uma busca aleatória para otimização de hiperparâmetros.

        :return: O melhor estimador encontrado ou None em caso de erro.
        """
        try:
            print("\n" + "╔" + "═" * 48 + "╗")
            print("║ Iniciando Ajuste de Hiperparâmetros...         ║")
            print("╠" + "═" * 48 + "╣")

            start_time = time.time()

            random_search = RandomizedSearchCV(
                self.model,
                self.param_distributions,
                n_iter=self.n_iter,
                cv=self.k_folds,
                scoring='accuracy',
                n_jobs=-1,
                random_state=42
            )

            random_search.fit(self.X_train, self.y_train)

            end_time = time.time()
            duration = end_time - start_time

            print("║ Ajuste Concluído!                              ║")
            print("╠" + "═" * 48 + "╣")
            print(f"║ Tempo de Execução: {duration:.2f} segundos               ║")
            print("╠" + "═" * 48 + "╣")
            print("║ Melhores Parâmetros:                           ║")
            for param, value in random_search.best_params_.items():
                print(f"║ • {param}: {value:<39}")
            print("╠" + "═" * 48 + "╣")
            print(f"║ Melhor Score: {random_search.best_score_:.4f}  ║")
            print("╚" + "═" * 48 + "╝")

            return random_search.best_estimator_
        except Exception as e:
            print("╔" + "═" * 48 + "╗")
            print("║ Erro durante o Random Search:                 ║")
            print("╠" + "═" * 48 + "╣")
            print(f"║ {str(e):<46}║")
            print("╚" + "═" * 48 + "╝")
            return None

class ModelEvaluation:
    """
    Classe para avaliar o desempenho dos modelos.
    """

    def __init__(self, best_model, X_test, y_test):
        """
        Inicializa a classe com o melhor modelo, conjunto de teste e target de teste.

        :param best_model: Melhor modelo encontrado.
        :param X_test: Features de teste.
        :param y_test: Target de teste.
        """
        self.best_model = best_model
        self.X_test = X_test
        self.y_test = y_test

    def evaluate(self):
        """
        Avalia o modelo usando o conjunto de teste.
        """
        y_pred = self.best_model.predict(self.X_test)
        print(f"Accuracy: {accuracy_score(self.y_test, y_pred)}")
        print(classification_report(self.y_test, y_pred))

def print_menu():
    """
    Imprime o menu principal.
    """
    print("\n" + "=" * 50)
    print("║         ANÁLISE DE DADOS DO TITANIC         ║")
    print("=" * 50)
    print("║ 1. Realizar Análise Descritiva              ║")
    print("║ 2. Pré-processar os Dados                   ║")
    print("║ 3. Buscar Passageiros                       ║")
    print("║ 4. Treinar Modelos                          ║")
    print("║ 5. Ajustar Hiperparâmetros                  ║")
    print("║ 6. Avaliar Modelos                          ║")
    print("║ 7. Sair                                     ║")
    print("=" * 50)

def main():
    """
    Função principal que coordena o fluxo do programa.
    """
    url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
    data_prep = DataPreparation(url)
    X_train, X_test, y_train, y_test = None, None, None, None
    dt_model, rf_model = None, None
    best_dt, best_rf = None, None

    while True:
        print_menu()
        choice = input("Escolha uma opção (1-7): ")

        if choice == '1':
            print("\n" + "-" * 50)
            print("Realizando Análise Descritiva...")
            print("-" * 50)
            data_prep.descriptive_analysis()

        elif choice == '2':
            print("\n" + "-" * 50)
            print("Pré-processando os Dados...")
            print("-" * 50)
            X_train, X_test, y_train, y_test = data_prep.preprocess_data()
            print("✔ Dados pré-processados com sucesso!")

        elif choice == '3':
            print("\n" + "-" * 50)
            print("Buscando Passageiros...")
            print("-" * 50)
            passenger_search_menu(data_prep.df)

        elif choice == '4':
            print("\n" + "-" * 50)
            print("Treinando Modelos...")
            print("-" * 50)
            if X_train is not None and y_train is not None:
                model_train = ModelTraining(X_train, y_train)

                dt_model = model_train.decision_tree()
                print("✔ Decision Tree treinado com sucesso!")

                rf_model = model_train.random_forest()
                print("✔ Random Forest treinado com sucesso!")
            else:
                print("❌ Por favor, pré-processe os dados primeiro.")

        elif choice == '5':
            print("\n" + "-" * 50)
            print("Ajustando Hiperparâmetros...")
            print("-" * 50)
            if dt_model is not None and rf_model is not None:
                param_dist_dt = {
                    'max_depth': randint(1, 20),
                    'min_samples_split': randint(2, 11),
                    'min_samples_leaf': randint(1, 11)
                }

                tuning_dt = HyperparameterTuning(dt_model, param_dist_dt, X_train, y_train, k_folds=5, n_iter=25)
                best_dt = tuning_dt.random_search()

                param_dist_rf = {
                    'n_estimators': randint(100, 500),
                    'max_depth': randint(1, 30),
                    'min_samples_split': randint(2, 11),
                    'min_samples_leaf': randint(1, 11),
                    'max_features': ['sqrt', 'log2', None]
                }
                tuning_rf = HyperparameterTuning(rf_model, param_dist_rf, X_train, y_train, k_folds=5, n_iter=25)
                best_rf = tuning_rf.random_search()

                print("✔ Hiperparâmetros ajustados com sucesso!")
            else:
                print("❌ Por favor, treine os modelos primeiro.")

        elif choice == '6':
            print("\n" + "-" * 50)
            print("Avaliando Modelos...")
            print("-" * 50)
            if best_dt is not None and best_rf is not None and X_test is not None and y_test is not None:
                evaluation_dt = ModelEvaluation(best_dt, X_test, y_test)
                evaluation_rf = ModelEvaluation(best_rf, X_test, y_test)

                print("Decision Tree Performance:")
                evaluation_dt.evaluate()

                print("\nRandom Forest Performance:")
                evaluation_rf.evaluate()
            else:
                print("❌ Por favor, ajuste os hiperparâmetros dos modelos e certifique-se de que os dados de teste estão disponíveis.")

        elif choice == '7':
            print("\n" + "=" * 50)
            print("Obrigado por usar o Analisador de Dados do Titanic!")
            print("=" * 50)
            break

        else:
            print("\n❌ Opção inválida! Por favor, escolha novamente.")

        input("\nPressione Enter para continuar...")

if __name__ == "__main__":
    main()


║         ANÁLISE DE DADOS DO TITANIC         ║
║ 1. Realizar Análise Descritiva              ║
║ 2. Pré-processar os Dados                   ║
║ 3. Buscar Passageiros                       ║
║ 4. Treinar Modelos                          ║
║ 5. Ajustar Hiperparâmetros                  ║
║ 6. Avaliar Modelos                          ║
║ 7. Sair                                     ║
Escolha uma opção (1-7): 2

--------------------------------------------------
Pré-processando os Dados...
--------------------------------------------------
✔ Dados pré-processados com sucesso!

Pressione Enter para continuar...4

║         ANÁLISE DE DADOS DO TITANIC         ║
║ 1. Realizar Análise Descritiva              ║
║ 2. Pré-processar os Dados                   ║
║ 3. Buscar Passageiros                       ║
║ 4. Treinar Modelos                          ║
║ 5. Ajustar Hiperparâmetros                  ║
║ 6. Avaliar Modelos                          ║
║ 7. Sair                                     ║